In [1]:
from define import qcode as qc
import numpy as np
from define.QECCLfid import uncorrectable as uc
from define import globalvars as gv
from define import chanreps as chrep

In [27]:
def Kron(*mats):
	"""
	Kronecker product of a list of matrices.
	"""
	if len(mats) <= 1:
		return mats[0]
	return np.kron(mats[0], Kron(*mats[1:]))

In [52]:
qcode = qc.QuantumErrorCorrectingCode("Steane")
qc.Load(qcode)
qc.populate_symplectic(qcode)
uc.ComputeCorrectableIndices(qcode)
correctable_set = np.array(qcode.Paulis_correctable, dtype=np.int)
probs = np.random.rand(4)
probs[0] = 1-0.66**12
probs[1:] = (1-probs[0]) * probs[1:]/np.sum(probs[1:])
print(probs[0])

0.9931683245467525


In [46]:
def uncorr_l2(probs, qcode, correctable_set,degen_lst):
    # compute coset probabilities
    if probs.ndim == 2:
        pauli_probs = np.prod(probs[range(qcode.N), qcode.PauliOperatorsLST[:, range(qcode.N)]], axis=1)
    else:
        pauli_probs = probs
    print("sum(P) = {}\nP(I) = {}".format(sum(pauli_probs), pauli_probs[0]))
    nlogs = 4**qcode.K
    nstabs = 2**(qcode.N - qcode.K)
    coset_probs = np.zeros(nlogs,dtype=np.double)
    for i in range(nlogs):
#         inds = i*(nstabs**2) + range(nstabs)*nstabs
        inds = degen_lst[i]
        coset_probs[i] = np.sum(pauli_probs[inds])
        # cosets[i] = all errors that are mapped to Li by the decoder.
    # get level-2 contribution
    # Note not including all identity correctable because it is accounted for in the l1 case
    # correctable_prob_l2 = np.sum(list(map(lambda x: np.prod(coset_probs[x]),qcode.Paulis_correctable[1:])))
    correctable_prob_l2 = np.sum(np.prod(coset_probs[correctable_set[1:]], axis=1))
    # Get level-1 contribution
    correctable_prob_l1 = np.power(np.sum(pauli_probs[qcode.PauliCorrectableIndices]), qcode.N)
    #print("Level 1 correctable : {}".format(correctable_prob_l1))
    #print("Level 2 correctable : {}".format(correctable_prob_l2))
    print("pc = {}\nP1 = {}\nP2 = {} \nPc2 = {}".format(np.sum(pauli_probs[qcode.PauliCorrectableIndices]), correctable_prob_l1, correctable_prob_l2,correctable_prob_l2+correctable_prob_l1))
    return 1 - (correctable_prob_l1 + correctable_prob_l2)

In [53]:
uncorr_l2(np.tile(probs, [qcode.N, 1]), qcode, correctable_set,degen_lst)

sum(P) = 0.9999999999999183
P(I) = 0.9531472957057473
pc = 0.979483217562962
P1 = 0.8649260838031094
P2 = 0.12422504826297104 
Pc2 = 0.9891511320660804


0.010848867933919593

In [58]:
uncorr_l2(np.tile(probs, [qcode.N, 1]), qcode, correctable_set)

sum(P) = 1.0000000000000175
P(I) = 0.6983372960937497
pc = 0.8516230599166119
P1 = 0.3248866665074825
P2 = 0.00012867390095955058


0.674984659591558

In [ ]:
%load_ext line_profiler

In [12]:
%lprun -f uncorr_l2 uncorr_l2(probs,qcode, correctable_set)

Level 1 correctable : 0.5790407441137769
Level 2 correctable : 0.004541670635943977


In [6]:
probs = np.random.rand(qcode.N, 4)

In [8]:
pauli_probs = np.ones(qcode.PauliOperatorsLST.shape[0], dtype=np.double)
for i in range(qcode.PauliOperatorsLST.shape[0]):
    for q in range(qcode.N):
        pauli_probs[i] *= probs[q, qcode.PauliOperatorsLST[i, q]]
print(pauli_probs)

[2.55292603e-04 4.07366780e-04 2.09175626e-05 ... 2.73921432e-05
 5.33458384e-04 4.95317874e-04]


In [44]:
qcode.PauliCorrectableIndices

array([  0,  64, 128, ..., 383, 447, 511])

In [62]:
def pcorr_level(probs, nblocks, level, qcode):
    if level > 0:
        return np.power(np.sum(probs[qcode.PauliCorrectableIndices[1:]]), nblocks)
    return np.power(np.sum(probs[qcode.PauliCorrectableIndices]), nblocks)

In [83]:
def CorrectableProbs(prob_levels, qcode):
    pcorr = 0
    nlevels = prob_levels.shape[0]
    for l in range(nlevels):
        probs = prob_levels[l, :, :]
        if probs.ndim == 2:
            pauli_probs = np.prod(probs[range(qcode.N), qcode.PauliOperatorsLST[:, range(qcode.N)]], axis=1)
        else:
            pauli_probs = probs
        print("Fidelity for level {} = {}".format(l, pauli_probs[0]))
        pcorr += pcorr_level(pauli_probs, qcode.N**(nlevels-l-1), l, qcode)
        print("pc for level {}: {}".format(l, pcorr_level(pauli_probs, qcode.N**(nlevels-l-1), l, qcode)))
    print("pc = {}".format(pcorr))
    return pcorr

In [90]:
fids = [0.99, 0.999]
nlevels = 2
probs = np.random.rand(2, 4)
# Level 1
probs[0, 0] = fids[0]
probs[0, 1:] = (1 - fids[0]) * probs[0, 1:]/np.sum(probs[0, 1:])
# Level 2
probs[1, 0] = fids[1]
probs[1, 1:] = (1 - fids[1]) * probs[1, 1:]/np.sum(probs[1, 1:])

prob_levels = np.zeros((nlevels, qcode.N, 4))
prob_levels[0, :, :] = np.tile(probs[0, :], [qcode.N, 1])
prob_levels[1, :, :] = np.tile(probs[1, :], [qcode.N, 1])

In [91]:
CorrectableProbs(prob_levels, qcode)

Fidelity for level 0 = 0.9320653479069899
pc for level 0: 0.8077407822403662
Fidelity for level 1 = 0.993020965034979
pc for level 1: 0.003978751318850704
pc = 0.8117195335592169


0.8117195335592169

In [92]:
logchans = np.load("/Users/pavi/Documents/chbank/23_05_2020_04_17_27/channels/logchan_8.5_-1_s306.npy")

In [104]:
np.diag(chrep.ConvertRepresentations(logchans[1, :, :], "process", "chi"))

array([9.99827018e-01+0.j, 4.06618895e-05+0.j, 7.29414150e-05+0.j,
       5.93790502e-05+0.j])

In [99]:
physchans = np.load("/Users/pavi/Documents/chbank/23_05_2020_04_17_27/physical/raw_rtasu_8.5_-1.npy")[306, :].reshape([7,4,4])

In [110]:
prob_levels[0, :, :] = np.array(list(map(lambda ls: np.real(np.diag(ls)), physchans)))

In [111]:
prob_levels[1, :, :] = np.tile(np.real(np.diag(chrep.ConvertRepresentations(logchans[1, :, :], "process", "chi"))), [qcode.N, 1])

In [112]:
prob_levels

array([[[9.95948296e-01, 6.28067740e-05, 2.51583404e-03, 1.47306271e-03],
        [9.90511794e-01, 1.42979216e-03, 1.59150636e-06, 8.05682248e-03],
        [9.99143793e-01, 2.11396638e-04, 6.43298907e-04, 1.51139545e-06],
        [9.98090970e-01, 8.29513533e-04, 7.15897990e-04, 3.63618508e-04],
        [9.95520511e-01, 4.17155609e-03, 3.06951643e-04, 9.81489786e-07],
        [9.99381151e-01, 1.15111616e-04, 3.79734883e-04, 1.24002699e-04],
        [9.98064598e-01, 6.04745575e-07, 1.93467641e-03, 1.20983173e-07]],

       [[9.99827018e-01, 4.06618895e-05, 7.29414150e-05, 5.93790502e-05],
        [9.99827018e-01, 4.06618895e-05, 7.29414150e-05, 5.93790502e-05],
        [9.99827018e-01, 4.06618895e-05, 7.29414150e-05, 5.93790502e-05],
        [9.99827018e-01, 4.06618895e-05, 7.29414150e-05, 5.93790502e-05],
        [9.99827018e-01, 4.06618895e-05, 7.29414150e-05, 5.93790502e-05],
        [9.99827018e-01, 4.06618895e-05, 7.29414150e-05, 5.93790502e-05],
        [9.99827018e-01, 4.06618895e

In [114]:
CorrectableProbs(prob_levels[:1, :, :], qcode)

Fidelity for level 0 = 0.9768650742956563
pc for level 0: 0.992912439778069
pc = 0.992912439778069


0.992912439778069

In [27]:
nstabs =  2**(qcode.N - qcode.K)
degen = {}
for l in range(4):
    degen[l], = np.nonzero(qcode.lookup[:,0]==l)
degen_lst = {}
for l in range(4):
    degen_lst[l] = np.array(list(map(lambda t : l*(nstabs**2) + range(nstabs)*nstabs + t,degen[l])),dtype=np.int).ravel()

In [29]:
degen_lst

{0: array([   0,   64,  128,  192,  256,  320,  384,  448,  512,  576,  640,
         704,  768,  832,  896,  960, 1024, 1088, 1152, 1216, 1280, 1344,
        1408, 1472, 1536, 1600, 1664, 1728, 1792, 1856, 1920, 1984, 2048,
        2112, 2176, 2240, 2304, 2368, 2432, 2496, 2560, 2624, 2688, 2752,
        2816, 2880, 2944, 3008, 3072, 3136, 3200, 3264, 3328, 3392, 3456,
        3520, 3584, 3648, 3712, 3776, 3840, 3904, 3968, 4032,    4,   68,
         132,  196,  260,  324,  388,  452,  516,  580,  644,  708,  772,
         836,  900,  964, 1028, 1092, 1156, 1220, 1284, 1348, 1412, 1476,
        1540, 1604, 1668, 1732, 1796, 1860, 1924, 1988, 2052, 2116, 2180,
        2244, 2308, 2372, 2436, 2500, 2564, 2628, 2692, 2756, 2820, 2884,
        2948, 3012, 3076, 3140, 3204, 3268, 3332, 3396, 3460, 3524, 3588,
        3652, 3716, 3780, 3844, 3908, 3972, 4036,    5,   69,  133,  197,
         261,  325,  389,  453,  517,  581,  645,  709,  773,  837,  901,
         965, 1029, 1093, 1157, 122

In [21]:
degen_lst

{0: [array([   0,   64,  128,  192,  256,  320,  384,  448,  512,  576,  640,
          704,  768,  832,  896,  960, 1024, 1088, 1152, 1216, 1280, 1344,
         1408, 1472, 1536, 1600, 1664, 1728, 1792, 1856, 1920, 1984, 2048,
         2112, 2176, 2240, 2304, 2368, 2432, 2496, 2560, 2624, 2688, 2752,
         2816, 2880, 2944, 3008, 3072, 3136, 3200, 3264, 3328, 3392, 3456,
         3520, 3584, 3648, 3712, 3776, 3840, 3904, 3968, 4032]),
  array([   4,   68,  132,  196,  260,  324,  388,  452,  516,  580,  644,
          708,  772,  836,  900,  964, 1028, 1092, 1156, 1220, 1284, 1348,
         1412, 1476, 1540, 1604, 1668, 1732, 1796, 1860, 1924, 1988, 2052,
         2116, 2180, 2244, 2308, 2372, 2436, 2500, 2564, 2628, 2692, 2756,
         2820, 2884, 2948, 3012, 3076, 3140, 3204, 3268, 3332, 3396, 3460,
         3524, 3588, 3652, 3716, 3780, 3844, 3908, 3972, 4036]),
  array([   5,   69,  133,  197,  261,  325,  389,  453,  517,  581,  645,
          709,  773,  837,  901,  965, 102